- https://python.langchain.com/docs/modules/agents/quick_start

In [ ]:
# Replace the following function _to_args_and_kwargs in \\Python311\Lib\site-packages\langchain_core\tools.py with the code below:

def _to_args_and_kwargs(self, tool_input: Union[str, Dict]) -> Tuple[Tuple, Dict]:
        """Convert tool input to pydantic model."""
        args, kwargs = super()._to_args_and_kwargs(tool_input)
        # For backwards compatibility. The tool must be run with a single input
        all_args = list(args) + list(kwargs.values())
        return tuple(all_args), {}

In [1]:
from langchain.agents import AgentType, initialize_agent
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint="",
    openai_api_key="",
    openai_api_type="azure",
    openai_api_version="2023-07-01-preview",
    deployment_name="gpt-4-32k",
    temperature=0,
)

In [2]:
class ExtentInput(BaseModel):
    lat: float = Field(description="A latitude value")
    lon: float = Field(description="A longitude value")
    dist: float = Field(description="The extent width, height or distance")
    pass

def calculate_extent(lat: float, lon: float, dist: float) -> str:
    # Simplistic calculation of extent (not geographically accurate)
    min_lat = lat - dist
    max_lat = lat + dist
    min_lon = lon - dist
    max_lon = lon + dist
    return f"Extent: ({min_lat}, {min_lon}), ({max_lat}, {max_lon})"

async def acalculate_extent(lat: float, lon: float, dist: float) -> str:
    # Async version of the calculate_extent function
    return calculate_extent(lat, lon, dist)

tools = [
    Tool(
        name="CalculateExtent",
        func=calculate_extent,
        description="A tool that calculates an extent around a given point.",
        args_schema=ExtentInput,
        coroutine=acalculate_extent,
        return_direct=True,
    )
]

In [3]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [4]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [5]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [6]:
question = "Calculate an extent of 2.4 degress around Beirut"

resp = agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...

Invoking: `CalculateExtent` with `{'lat': 33.8938, 'lon': 35.5018, 'dist': 2.4}`


Extent: (31.4938, 33.101800000000004), (36.2938, 37.9018)


> Finished chain.
